## State Population and Vaccination Data

In [1]:
# Import Pandas, Config Variables, and SQLalchemy
import pandas as pd
from config import endpoint, username, password
from sqlalchemy import create_engine

In [2]:
# Read CSVs for State Data
state_pop_df = pd.read_csv("../data/state_populations_cleaned.csv")
state_vacc_df = pd.read_csv("../data/state_covid_vaccinations_cleaned.csv")
state_latlons_df = pd.read_csv("../data/statelatlong.csv")

In [3]:
# State Populations
state_pop_df.head()

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223


In [4]:
# State Vaccinations
state_vacc_df.head()

,state,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alaska,299935,235141,41000.0,32143.0,152349,20826,82272,11246
1,Alabama,1082100,788613,22069.0,16084.0,567505,11574,219639,4480
2,Arkansas,691525,528467,22915.0,17512.0,367436,12176,157511,5219
3,American Samoa,29450,18778,52883.0,33719.0,10948,19659,7811,14026
4,Arizona,1688255,1469676,23194.0,20191.0,1087921,14947,376371,5171


In [5]:
state_latlons_df.head()

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [6]:
state_latlons_df = state_latlons_df.rename(columns={"State": "Abb", "City": "state"})

In [7]:
state_latlons_df.head()

,Abb,Latitude,Longitude,state
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [8]:
# State Data Merged
state_merged_df = pd.merge(state_pop_df, state_vacc_df, how="inner", on="state")
state_merged_df.head()

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alabama,4903185,1082100,788613,22069.0,16084.0,567505,11574,219639,4480
1,Alaska,731545,299935,235141,41000.0,32143.0,152349,20826,82272,11246
2,Arizona,7278717,1688255,1469676,23194.0,20191.0,1087921,14947,376371,5171
3,Arkansas,3017804,691525,528467,22915.0,17512.0,367436,12176,157511,5219
4,California,39512223,9178020,7453468,23228.0,18864.0,5426591,13734,1974488,4997


In [9]:
state_merged_df = pd.merge(state_merged_df, state_latlons_df, how="left", on="state")

In [10]:
state_merged_df.head()

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude
0,Alabama,4903185,1082100,788613,22069.0,16084.0,567505,11574,219639,4480,AL,32.601011,-86.680736
1,Alaska,731545,299935,235141,41000.0,32143.0,152349,20826,82272,11246,AK,61.302501,-158.775020
2,Arizona,7278717,1688255,1469676,23194.0,20191.0,1087921,14947,376371,5171,AZ,34.168219,-111.930907
3,Arkansas,3017804,691525,528467,22915.0,17512.0,367436,12176,157511,5219,AR,34.751928,-92.131378
4,California,39512223,9178020,7453468,23228.0,18864.0,5426591,13734,1974488,4997,CA,37.271875,-119.270415


In [11]:
# Set State as the Index
state_info_df = state_merged_df.set_index("state") 
state_info_df.head()

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude
state,,,,,,,,,,,,
Alabama,4903185,1082100,788613,22069.0,16084.0,567505,11574,219639,4480,AL,32.601011,-86.680736
Alaska,731545,299935,235141,41000.0,32143.0,152349,20826,82272,11246,AK,61.302501,-158.775020
Arizona,7278717,1688255,1469676,23194.0,20191.0,1087921,14947,376371,5171,AZ,34.168219,-111.930907
Arkansas,3017804,691525,528467,22915.0,17512.0,367436,12176,157511,5219,AR,34.751928,-92.131378
California,39512223,9178020,7453468,23228.0,18864.0,5426591,13734,1974488,4997,CA,37.271875,-119.270415


In [12]:
# Add Empty Column for New Calculation
state_info_df["percent_vaccinated"] = ""
state_info_df.head(10)

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude,percent_vaccinated
state,,,,,,,,,,,,,
Alabama,4903185,1082100,788613,22069.0,16084.0,567505,11574,219639,4480,AL,32.601011,-86.680736,
Alaska,731545,299935,235141,41000.0,32143.0,152349,20826,82272,11246,AK,61.302501,-158.775020,
Arizona,7278717,1688255,1469676,23194.0,20191.0,1087921,14947,376371,5171,AZ,34.168219,-111.930907,
Arkansas,3017804,691525,528467,22915.0,17512.0,367436,12176,157511,5219,AR,34.751928,-92.131378,
California,39512223,9178020,7453468,23228.0,18864.0,5426591,13734,1974488,4997,CA,37.271875,-119.270415,
Colorado,5758736,1348800,1202161,23422.0,20875.0,817009,14187,378242,6568,CO,38.997934,-105.550567,
Connecticut,3565287,984275,860534,27607.0,24136.0,579380,16251,270792,7595,CT,41.518784,-72.757507,
Delaware,973764,204200,180940,20970.0,18582.0,133251,13684,46803,4806,DE,39.145251,-75.418921,
District of Columbia,705749,225125,162674,31899.0,23050.0,109045,15451,53244,7544,DC,38.899349,-77.014567,


In [13]:
# Calcuation for percent vaccinated by state
for index, row in state_info_df.iterrows():
    population = row["population"]
    vaccinated = row["total_administered"]
    percent_vaccinated = round((vaccinated / population) * 100, 2)
    
    state_info_df.at[index, "percent_vaccinated"] = percent_vaccinated

In [14]:
# Check the data types
state_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, Alabama to Wyoming
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   population                        51 non-null     int64  
 1   total_distributed                 51 non-null     int64  
 2   total_administered                51 non-null     int64  
 3   distributed_per_100K              51 non-null     float64
 4   administered_per_100K             51 non-null     float64
 5   people_with_1plus_doses           51 non-null     int64  
 6   people_with_1plus_doses_per_100K  51 non-null     int64  
 7   people_with_2_doses               51 non-null     int64  
 8   people_with_2_doses_per_100K      51 non-null     int64  
 9   Abb                               51 non-null     object 
 10  Latitude                          51 non-null     float64
 11  Longitude                         51 non-null     float64
 12  perc

In [15]:
# Convert percent vaccinated into a float
state_info_df = state_info_df.astype({"percent_vaccinated": 'float64'})
state_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, Alabama to Wyoming
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   population                        51 non-null     int64  
 1   total_distributed                 51 non-null     int64  
 2   total_administered                51 non-null     int64  
 3   distributed_per_100K              51 non-null     float64
 4   administered_per_100K             51 non-null     float64
 5   people_with_1plus_doses           51 non-null     int64  
 6   people_with_1plus_doses_per_100K  51 non-null     int64  
 7   people_with_2_doses               51 non-null     int64  
 8   people_with_2_doses_per_100K      51 non-null     int64  
 9   Abb                               51 non-null     object 
 10  Latitude                          51 non-null     float64
 11  Longitude                         51 non-null     float64
 12  perc

In [16]:
state_info_df.head(10)

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude,percent_vaccinated
state,,,,,,,,,,,,,
Alabama,4903185,1082100,788613,22069.0,16084.0,567505,11574,219639,4480,AL,32.601011,-86.680736,16.08
Alaska,731545,299935,235141,41000.0,32143.0,152349,20826,82272,11246,AK,61.302501,-158.775020,32.14
Arizona,7278717,1688255,1469676,23194.0,20191.0,1087921,14947,376371,5171,AZ,34.168219,-111.930907,20.19
Arkansas,3017804,691525,528467,22915.0,17512.0,367436,12176,157511,5219,AR,34.751928,-92.131378,17.51
California,39512223,9178020,7453468,23228.0,18864.0,5426591,13734,1974488,4997,CA,37.271875,-119.270415,18.86
Colorado,5758736,1348800,1202161,23422.0,20875.0,817009,14187,378242,6568,CO,38.997934,-105.550567,20.88
Connecticut,3565287,984275,860534,27607.0,24136.0,579380,16251,270792,7595,CT,41.518784,-72.757507,24.14
Delaware,973764,204200,180940,20970.0,18582.0,133251,13684,46803,4806,DE,39.145251,-75.418921,18.58
District of Columbia,705749,225125,162674,31899.0,23050.0,109045,15451,53244,7544,DC,38.899349,-77.014567,23.05


## NIH AWS Data / NYT

### Set up State Master DF

In [17]:
# Read NYT / NIH json 
nytimes_nih_covid_df = pd.read_json("../data/daily_covid_stats_by_state_nyt.json", lines=True)

In [18]:
# Get Data for Most Recent Provided Date
most_recent_date = "2021-02-22"
latest_nyt_stateData = nytimes_nih_covid_df[nytimes_nih_covid_df['date'] == f'{most_recent_date}']

In [19]:
latest_nyt_stateData.head(10)

,date,state,fips,cases,deaths
19594,2021-02-22,Alabama,1,487520,9592
19595,2021-02-22,Alaska,2,57316,279
19596,2021-02-22,Arizona,4,809899,15502
19597,2021-02-22,Arkansas,5,315759,5363
19598,2021-02-22,California,6,3536446,49531
19599,2021-02-22,Colorado,8,424097,5990
19600,2021-02-22,Connecticut,9,275334,7562
19601,2021-02-22,Delaware,10,85090,1368
19602,2021-02-22,District of Columbia,11,39755,995
19603,2021-02-22,Florida,12,1872915,30064


In [20]:
import datetime as dt

for index, row in nytimes_nih_covid_df.iterrows():
    date = row["date"].strftime('%Y-%m-%d')
    nytimes_nih_covid_df.at[index, "date"] = date

In [21]:
nytimes_nih_covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19649 entries, 0 to 19648
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    19649 non-null  object
 1   state   19649 non-null  object
 2   fips    19649 non-null  int64 
 3   cases   19649 non-null  int64 
 4   deaths  19649 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 767.7+ KB


In [22]:
# Merge with State Info DF that contains population and vaccinations
latest_data_merged = pd.merge(state_info_df, latest_nyt_stateData, on="state")
latest_data_merged.head(10)

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude,percent_vaccinated,date,fips,cases,deaths
0,Alabama,4903185,1082100,788613,22069.0,16084.0,567505,11574,219639,4480,AL,32.601011,-86.680736,16.08,2021-02-22,1,487520,9592
1,Alaska,731545,299935,235141,41000.0,32143.0,152349,20826,82272,11246,AK,61.302501,-158.775020,32.14,2021-02-22,2,57316,279
2,Arizona,7278717,1688255,1469676,23194.0,20191.0,1087921,14947,376371,5171,AZ,34.168219,-111.930907,20.19,2021-02-22,4,809899,15502
3,Arkansas,3017804,691525,528467,22915.0,17512.0,367436,12176,157511,5219,AR,34.751928,-92.131378,17.51,2021-02-22,5,315759,5363
4,California,39512223,9178020,7453468,23228.0,18864.0,5426591,13734,1974488,4997,CA,37.271875,-119.270415,18.86,2021-02-22,6,3536446,49531
5,Colorado,5758736,1348800,1202161,23422.0,20875.0,817009,14187,378242,6568,CO,38.997934,-105.550567,20.88,2021-02-22,8,424097,5990
6,Connecticut,3565287,984275,860534,27607.0,24136.0,579380,16251,270792,7595,CT,41.518784,-72.757507,24.14,2021-02-22,9,275334,7562
7,Delaware,973764,204200,180940,20970.0,18582.0,133251,13684,46803,4806,DE,39.145251,-75.418921,18.58,2021-02-22,10,85090,1368
8,District of Columbia,705749,225125,162674,31899.0,23050.0,109045,15451,53244,7544,DC,38.899349,-77.014567,23.05,2021-02-22,11,39755,995
9,Florida,21477737,5120975,4387051,23843.0,20426.0,2877011,13395,1481148,6896,FL,27.975728,-83.833017,20.43,2021-02-22,12,1872915,30064


In [23]:
# Create a Master Dataframe to start working with
state_master_df = latest_data_merged[['state', 'Abb', 'date', 'population', 'Latitude', 'Longitude', 'cases', 'deaths', 'total_distributed', 'total_administered', 'percent_vaccinated' ]]
state_master_df.head(10)

,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated
0,Alabama,AL,2021-02-22,4903185,32.601011,-86.680736,487520,9592,1082100,788613,16.08
1,Alaska,AK,2021-02-22,731545,61.302501,-158.775020,57316,279,299935,235141,32.14
2,Arizona,AZ,2021-02-22,7278717,34.168219,-111.930907,809899,15502,1688255,1469676,20.19
3,Arkansas,AR,2021-02-22,3017804,34.751928,-92.131378,315759,5363,691525,528467,17.51
4,California,CA,2021-02-22,39512223,37.271875,-119.270415,3536446,49531,9178020,7453468,18.86
5,Colorado,CO,2021-02-22,5758736,38.997934,-105.550567,424097,5990,1348800,1202161,20.88
6,Connecticut,CT,2021-02-22,3565287,41.518784,-72.757507,275334,7562,984275,860534,24.14
7,Delaware,DE,2021-02-22,973764,39.145251,-75.418921,85090,1368,204200,180940,18.58
8,District of Columbia,DC,2021-02-22,705749,38.899349,-77.014567,39755,995,225125,162674,23.05
9,Florida,FL,2021-02-22,21477737,27.975728,-83.833017,1872915,30064,5120975,4387051,20.43


In [24]:
# Create empty column to calculate estimated percent infected to date
state_master_df["est_percent_infected_to_date"] = ""
state_master_df.head(10)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date
0,Alabama,AL,2021-02-22,4903185,32.601011,-86.680736,487520,9592,1082100,788613,16.08,
1,Alaska,AK,2021-02-22,731545,61.302501,-158.775020,57316,279,299935,235141,32.14,
2,Arizona,AZ,2021-02-22,7278717,34.168219,-111.930907,809899,15502,1688255,1469676,20.19,
3,Arkansas,AR,2021-02-22,3017804,34.751928,-92.131378,315759,5363,691525,528467,17.51,
4,California,CA,2021-02-22,39512223,37.271875,-119.270415,3536446,49531,9178020,7453468,18.86,
5,Colorado,CO,2021-02-22,5758736,38.997934,-105.550567,424097,5990,1348800,1202161,20.88,
6,Connecticut,CT,2021-02-22,3565287,41.518784,-72.757507,275334,7562,984275,860534,24.14,
7,Delaware,DE,2021-02-22,973764,39.145251,-75.418921,85090,1368,204200,180940,18.58,
8,District of Columbia,DC,2021-02-22,705749,38.899349,-77.014567,39755,995,225125,162674,23.05,
9,Florida,FL,2021-02-22,21477737,27.975728,-83.833017,1872915,30064,5120975,4387051,20.43,


In [25]:
# Calcuation for est percent infected by state
for index, row in state_master_df.iterrows():
    population = row["population"]
    deaths = row["deaths"]
    infected = row["cases"]
    
    # take out the number who have passed away to reset estimated population
    est_population_new = population - deaths 
    
    # estimate the percent infected
    percent_infected = round((infected / est_population_new) * 100, 2)
    
    # insert into percent_infected column
    state_master_df.at[index, "est_percent_infected_to_date"] = percent_infected

In [26]:
state_master_df.head(10)

,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date
0,Alabama,AL,2021-02-22,4903185,32.601011,-86.680736,487520,9592,1082100,788613,16.08,9.96
1,Alaska,AK,2021-02-22,731545,61.302501,-158.775020,57316,279,299935,235141,32.14,7.84
2,Arizona,AZ,2021-02-22,7278717,34.168219,-111.930907,809899,15502,1688255,1469676,20.19,11.15
3,Arkansas,AR,2021-02-22,3017804,34.751928,-92.131378,315759,5363,691525,528467,17.51,10.48
4,California,CA,2021-02-22,39512223,37.271875,-119.270415,3536446,49531,9178020,7453468,18.86,8.96
5,Colorado,CO,2021-02-22,5758736,38.997934,-105.550567,424097,5990,1348800,1202161,20.88,7.37
6,Connecticut,CT,2021-02-22,3565287,41.518784,-72.757507,275334,7562,984275,860534,24.14,7.74
7,Delaware,DE,2021-02-22,973764,39.145251,-75.418921,85090,1368,204200,180940,18.58,8.75
8,District of Columbia,DC,2021-02-22,705749,38.899349,-77.014567,39755,995,225125,162674,23.05,5.64
9,Florida,FL,2021-02-22,21477737,27.975728,-83.833017,1872915,30064,5120975,4387051,20.43,8.73


In [27]:
# Add Estimated Percent Immune Column
state_master_df["est_percent_immune"] = ""
state_master_df.head(10)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date,est_percent_immune
0,Alabama,AL,2021-02-22,4903185,32.601011,-86.680736,487520,9592,1082100,788613,16.08,9.96,
1,Alaska,AK,2021-02-22,731545,61.302501,-158.775020,57316,279,299935,235141,32.14,7.84,
2,Arizona,AZ,2021-02-22,7278717,34.168219,-111.930907,809899,15502,1688255,1469676,20.19,11.15,
3,Arkansas,AR,2021-02-22,3017804,34.751928,-92.131378,315759,5363,691525,528467,17.51,10.48,
4,California,CA,2021-02-22,39512223,37.271875,-119.270415,3536446,49531,9178020,7453468,18.86,8.96,
5,Colorado,CO,2021-02-22,5758736,38.997934,-105.550567,424097,5990,1348800,1202161,20.88,7.37,
6,Connecticut,CT,2021-02-22,3565287,41.518784,-72.757507,275334,7562,984275,860534,24.14,7.74,
7,Delaware,DE,2021-02-22,973764,39.145251,-75.418921,85090,1368,204200,180940,18.58,8.75,
8,District of Columbia,DC,2021-02-22,705749,38.899349,-77.014567,39755,995,225125,162674,23.05,5.64,
9,Florida,FL,2021-02-22,21477737,27.975728,-83.833017,1872915,30064,5120975,4387051,20.43,8.73,


In [28]:
# Calcuation for estimated immune by state
for index, row in state_master_df.iterrows():
    population = row["population"]
    infected = row["cases"]
    deaths = row["deaths"]
    vaccinated = row["total_administered"]
    
    est_total_immune = infected + vaccinated
    est_population = population - deaths
    
    percent_immune = round((est_total_immune / est_population) * 100, 2)
    
    state_master_df.at[index, "est_percent_immune"] = percent_immune

In [29]:
state_master_df.head(10)

,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date,est_percent_immune
0,Alabama,AL,2021-02-22,4903185,32.601011,-86.680736,487520,9592,1082100,788613,16.08,9.96,26.08
1,Alaska,AK,2021-02-22,731545,61.302501,-158.775020,57316,279,299935,235141,32.14,7.84,39.99
2,Arizona,AZ,2021-02-22,7278717,34.168219,-111.930907,809899,15502,1688255,1469676,20.19,11.15,31.39
3,Arkansas,AR,2021-02-22,3017804,34.751928,-92.131378,315759,5363,691525,528467,17.51,10.48,28.02
4,California,CA,2021-02-22,39512223,37.271875,-119.270415,3536446,49531,9178020,7453468,18.86,8.96,27.85
5,Colorado,CO,2021-02-22,5758736,38.997934,-105.550567,424097,5990,1348800,1202161,20.88,7.37,28.27
6,Connecticut,CT,2021-02-22,3565287,41.518784,-72.757507,275334,7562,984275,860534,24.14,7.74,31.93
7,Delaware,DE,2021-02-22,973764,39.145251,-75.418921,85090,1368,204200,180940,18.58,8.75,27.36
8,District of Columbia,DC,2021-02-22,705749,38.899349,-77.014567,39755,995,225125,162674,23.05,5.64,28.72
9,Florida,FL,2021-02-22,21477737,27.975728,-83.833017,1872915,30064,5120975,4387051,20.43,8.73,29.19


### Set up Daily Cases DF

In [30]:
nyt_avg_daily_cases = nytimes_nih_covid_df
nyt_avg_daily_cases.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [31]:
nyt_avg_daily_cases = nyt_avg_daily_cases.groupby("date").sum()
nyt_avg_daily_cases.head()

,fips,cases,deaths
date,,,
2020-01-21,53,1,0
2020-01-22,53,1,0
2020-01-23,53,1,0
2020-01-24,70,2,0
2020-01-25,76,3,0


In [32]:
nyt_avg_daily_cases["daily_new_cases"] = ""

In [33]:
nyt_avg_daily_cases.head()

,fips,cases,deaths,daily_new_cases
date,,,,
2020-01-21,53,1,0,
2020-01-22,53,1,0,
2020-01-23,53,1,0,
2020-01-24,70,2,0,
2020-01-25,76,3,0,


In [34]:
previous_day = 0

In [35]:
for index, row in nyt_avg_daily_cases.iterrows():
    new_cases = row["cases"] - previous_day
    previous_day = row["cases"]
    nyt_avg_daily_cases.at[index, "daily_new_cases"] = new_cases

In [36]:
nyt_avg_daily_cases["date"] = ""

In [37]:
nyt_avg_daily_cases

,fips,cases,deaths,daily_new_cases,date
date,,,,,
2020-01-21,53,1,0,1,
2020-01-22,53,1,0,0,
2020-01-23,53,1,0,0,
2020-01-24,70,2,0,1,
2020-01-25,76,3,0,1,
...,...,...,...,...,...
2021-02-18,1762,27944914,492946,71874,
2021-02-19,1762,28022932,495572,78018,
2021-02-20,1762,28092672,497403,69740,


In [38]:
for index, row in nyt_avg_daily_cases.iterrows():
    date = index
    nyt_avg_daily_cases.at[index, "date"] = date

In [39]:
nyt_avg_daily_cases.reset_index(drop=True, inplace=True)

In [40]:
nyt_avg_daily_cases.head(10)

,fips,cases,deaths,daily_new_cases,date
0,53,1,0,1,2020-01-21
1,53,1,0,0,2020-01-22
2,53,1,0,0,2020-01-23
3,70,2,0,1,2020-01-24
4,76,3,0,1,2020-01-25
5,80,5,0,2,2020-01-26
6,80,5,0,0,2020-01-27
7,80,5,0,0,2020-01-28
8,80,5,0,0,2020-01-29
9,80,6,0,1,2020-01-30


### Set Up NYT Totals Master DF

In [41]:
nyt_master = pd.merge(nytimes_nih_covid_df, state_pop_df, how ="inner", on="state")
nyt_master.head()

,date,state,fips,cases,deaths,population
0,2020-01-21,Washington,53,1,0,7614893
1,2020-01-22,Washington,53,1,0,7614893
2,2020-01-23,Washington,53,1,0,7614893
3,2020-01-24,Washington,53,1,0,7614893
4,2020-01-25,Washington,53,1,0,7614893


In [42]:
nyt_master["est_percent_infected"] = ""
nyt_master.head()

,date,state,fips,cases,deaths,population,est_percent_infected
0,2020-01-21,Washington,53,1,0,7614893,
1,2020-01-22,Washington,53,1,0,7614893,
2,2020-01-23,Washington,53,1,0,7614893,
3,2020-01-24,Washington,53,1,0,7614893,
4,2020-01-25,Washington,53,1,0,7614893,


In [43]:
# Calcuation for est percent infected by state
for index, row in nyt_master.iterrows():
    population = int(row["population"])
    deaths = row["deaths"]
    infected = row["cases"]
    
    # take out the number who have passed away to reset estimated population
    est_population_new = population - deaths 
    
    # estimate the percent infected
    if infected == 0:
        percent_infected = 0
    else: 
        percent_infected = round((infected / est_population_new) * 100, 2)
    
    # insert into percent_infected column
    nyt_master.at[index, "est_percent_infected"] = percent_infected

In [44]:
nyt_master.head()

,date,state,fips,cases,deaths,population,est_percent_infected
0,2020-01-21,Washington,53,1,0,7614893,0
1,2020-01-22,Washington,53,1,0,7614893,0
2,2020-01-23,Washington,53,1,0,7614893,0
3,2020-01-24,Washington,53,1,0,7614893,0
4,2020-01-25,Washington,53,1,0,7614893,0


## JSON Exports

In [45]:
# Master DF to JSON
state_master_df.to_json("../data/state-master-data.json", orient="records")

In [46]:
# NYT Master to JSON 
nyt_master.to_json("../data/nyt-master.json", orient="records")

In [47]:
nyt_avg_daily_cases.to_json("../data/daily_new_cases.json", orient="records")

### CSV Exports

In [48]:
state_master_df.to_csv("../data/state-master-data.csv")

## Amazon RDS Updates

In [49]:
# Connect to AWS Database instance 
engine = create_engine(f'postgresql://uscovid:{password}@{endpoint}/us_covid_db')
connection = engine.connect()

In [50]:
# 50 States & D.C. Data
state_master_df.to_sql('master_table', index=False, if_exists='replace', con=connection)

In [51]:
# Daily Cases Data
nyt_avg_daily_cases.to_sql("daily_new_cases", index=False, if_exists='replace', con=connection)

In [ ]:
# Raw NTY Data
nytimes_nih_covid_df.to_sql('nyt_table', index=True, if_exists='replace', con=connection)